In [ ]:
from flask import Flask, render_template, jsonify, request
from src.helper import download_hugging_face_embeddings
from langchain_pinecone import PineconeVectorStore
from langchain_mistralai import ChatMistralAI
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

# Initialize Flask app
app = Flask(__name__)

# Load environment variables
load_dotenv("APIKey.env")

# Get API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# Ensure API keys exist
if not PINECONE_API_KEY or not MISTRAL_API_KEY:
    print("Missing API keys! Check your .env file.")
    exit(1)

# Download embeddings
embeddings = download_hugging_face_embeddings()

# Define Pinecone index name
index_name = "medicalbot"

# Load existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Initialize LLM
llm = ChatMistralAI(model="mistral-medium", temperature=0.1, max_output_tokens=200)
import mistral  # Ensure the correct Mistral SDK is installed
import os
from dotenv import load_dotenv

# Load API key
load_dotenv("APIKey.env")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")



# Improved system prompt
system_prompt = (
    "You are a highly knowledgeable and reliable AI assistant specializing in medical advice. "
    "Your responses should be clear, accurate, and based on trusted medical sources. "
    "Follow these guidelines while answering: "
    "1] Fact-Based Responses: Use only the provided context and reliable knowledge. If unsure, say you don't know. "
    "2] Concise & Professional: Provide answers in a structured, easy-to-understand format (3-5 sentences max). "
    "3] Avoid Diagnosis: You are not a doctor. Provide general guidance but advise users to consult a professional for medical concerns. "
    "4] Patient Safety First: Warn users about risks when discussing treatments, medications, or serious conditions. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create RAG chain
from langchain.chains.combine_documents import create_stuff_documents_chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

def get_mistral_response(user_input):
    response = mistral.chat(
        model="mistral-medium", 
        messages=[
            {"role": "system", "content": 
                "You are a medical assistant. Answer in a structured format with bullet points. "
                "Always use '-' before each point. Keep sentences short and clear.\n\n"
                "Example format:\n"
                "- Point 1\n"
                "- Point 2\n"
                "- Point 3"},
            {"role": "user", "content": user_input}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Flask Routes

@app.route("/")
def index():
    return render_template("chatbot.html")
    

@app.route("/get", methods=["POST"])
def chat():
    msg = request.form.get("msg", "").strip()

    if not msg:
        return jsonify("Empty message received!")

    print(f"User Input: {msg}")

    try:
        response = rag_chain.invoke({"input": msg})

        # Ensure response exists and contains a valid answer
        answer = response.get("answer", "").strip()

        if not answer or "error" in answer.lower():
            answer = "I'm sorry, but I don't know the answer to that."

        # Clean up response to remove unwanted characters
        answer = answer.replace("\n", " ").replace("\r", " ")
        answer = answer.replace("{", "").replace("}", "").replace('"', "")

        print(f"AI Response: {answer}")
        return jsonify(answer)

    except Exception as e:
        print(f"Error processing AI response: {str(e)}")
        return jsonify("An error occurred while fetching the response. Please try again.")

# Run Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.70.101.151:5000
Press CTRL+C to quit
172.70.101.151 - - [11/Apr/2025 17:31:46] "GET / HTTP/1.1" 200 -
172.70.101.151 - - [11/Apr/2025 17:31:47] "GET /favicon.ico HTTP/1.1" 404 -


User Input: i want to know about acne


172.70.101.151 - - [11/Apr/2025 17:32:07] "POST /get HTTP/1.1" 200 -


AI Response: Acne is a common skin condition that occurs when hair follicles become plugged with oil and dead skin cells. It often causes whiteheads, blackheads, or pimples, and mainly affects the face, forehead, chest, upper back, and shoulders. Acne is most common among teenagers, though it can affect people of all ages. While it's not a serious health threat, severe acne can cause emotional distress and lead to scarring.  To prevent acne, it's recommended to: 1. Cleanse the skin gently with a mild soap or cleanser twice a day. 2. Avoid picking or squeezing pimples, as this can lead to infection and scarring. 3. Keep hair clean and away from the face. 4. Avoid excessive sun exposure and use non-comedogenic (non-pore-clogging) skin care products and makeup. 5. In some cases, over-the-counter or prescription medications may be necessary to manage acne.  If you're experiencing persistent or severe acne, it's best to consult a healthcare professional or a dermatologist for proper evaluat